In [1]:
import pandas as pd

In [4]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import pymysql

In [5]:
df = pd.read_csv("C:/Users/user/Documents/서울시_포폴/법정동코드 전체자료.txt", encoding="cp949")

In [6]:
df

,법정동코드\t법정동명\t폐지여부
0,1100000000\t서울특별시\t존재
1,1111000000\t서울특별시 종로구\t존재
2,1111010100\t서울특별시 종로구 청운동\t존재
3,1111010200\t서울특별시 종로구 신교동\t존재
4,1111010300\t서울특별시 종로구 궁정동\t존재
...,...
49845,5280042024\t전북특별자치도 부안군 위도면 대리\t존재
49846,5280042025\t전북특별자치도 부안군 위도면 거륜리\t존재
49847,5280042026\t전북특별자치도 부안군 위도면 식도리\t존재
49848,5280042027\t전북특별자치도 부안군 위도면 상왕등리\t존재


In [14]:
# ✅ dim_admin_dong 데이터 적재용 최종 코드 (수정본)
# 0️⃣ 전제

# 파일: 법정동코드 전체자료.txt

# 구분자: \t

# 컬럼 구조: 법정동코드 / 법정동명 / 폐지여부

# fact 테이블의 행정동코드는 8자리

# dim 테이블 PK도 8자리

In [13]:
# 1️⃣ 파일 읽기 + 컬럼 강제 분리

In [7]:
import pandas as pd
import numpy as np
import pymysql

path = r"C:/Users/user/Documents/서울시_포폴/법정동코드 전체자료.txt"

# 1. 일단 그대로 읽기 (지금 상태 유지)
df = pd.read_csv(path, encoding="cp949")

# 2. 컬럼이 하나면 탭 기준으로 강제 분리
if len(df.columns) == 1:
    col = df.columns[0]
    df = df[col].astype(str).str.split("\t", expand=True)
    df.columns = ["법정동코드", "법정동명", "폐지여부"]

# 컬럼명 정리
df.columns = df.columns.str.strip()

df.head()

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재


In [15]:
# 2️⃣ 전처리 (서울 + 폐지 제외)

In [8]:
df = df.copy()

# 숫자형 변환
df["법정동코드"] = pd.to_numeric(df["법정동코드"], errors="coerce")
df = df.dropna(subset=["법정동코드", "법정동명"])

df["법정동코드"] = df["법정동코드"].astype("int64")
df["법정동명"] = df["법정동명"].astype(str).str.strip()

# 서울만
df = df[df["법정동명"].str.startswith("서울특별시")]

# 폐지 제외
df["폐지여부"] = df["폐지여부"].astype(str).str.strip()
df = df[df["폐지여부"] != "폐지"]

df.head()

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재


In [16]:
# 3️⃣ 🔑 10자리 → 8자리 행정동코드 생성 (핵심)

In [9]:
# 10자리 법정동코드 → 8자리 행정동코드
df["행정동코드"] = (
    df["법정동코드"]
    .astype(str)
    .str.slice(0, 8)
    .astype("int64")
)

In [17]:
# 4️⃣ 시도 / 구 / 동 분리

In [10]:
parts = df["법정동명"].str.split(r"\s+", expand=True)

df["시도명"] = parts[0]
df["시군구명"] = parts[1]
df["행정동명"] = parts[2]

df[["행정동코드","법정동코드","시도명","시군구명","행정동명"]].head()

,행정동코드,법정동코드,시도명,시군구명,행정동명
0,11000000,1100000000,서울특별시,None,None
1,11110000,1111000000,서울특별시,종로구,None
2,11110101,1111010100,서울특별시,종로구,청운동
3,11110102,1111010200,서울특별시,종로구,신교동
4,11110103,1111010300,서울특별시,종로구,궁정동


In [18]:
# 5️⃣ dim 테이블용 DataFrame 생성

In [11]:
dim = df[
    ["행정동코드", "법정동코드", "시도명", "시군구명", "행정동명"]
].copy()

dim["full_name"] = (
    dim["시도명"] + " " +
    dim["시군구명"] + " " +
    dim["행정동명"]
)

# 8자리 행정동코드 기준 중복 제거
dim = dim.drop_duplicates(subset=["행정동코드"])

dim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 493 entries, 0 to 1111
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   행정동코드      493 non-null    int64 
 1   법정동코드      493 non-null    int64 
 2   시도명        493 non-null    object
 3   시군구명       492 non-null    object
 4   행정동명       467 non-null    object
 5   full_name  467 non-null    object
dtypes: int64(2), object(4)
memory usage: 27.0+ KB


In [19]:
# 6️⃣ MySQL INSERT (pymysql)

In [20]:
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="********",
    db="seoul",
    port=3306,
    charset="utf8mb4",
    autocommit=False
)

sql = """
INSERT INTO dim_admin_dong
(행정동코드, 법정동코드, 시도명, 시군구명, 행정동명, full_name)
VALUES (%s,%s,%s,%s,%s,%s)
ON DUPLICATE KEY UPDATE
  법정동코드=VALUES(법정동코드),
  시도명=VALUES(시도명),
  시군구명=VALUES(시군구명),
  행정동명=VALUES(행정동명),
  full_name=VALUES(full_name)
"""

data = (
    dim.astype(object)
       .where(pd.notnull(dim), None)
       .values.tolist()
)

with conn.cursor() as cur:
    cur.executemany(sql, data)

conn.commit()
conn.close()

print("✅ dim_admin_dong 적재 완료:", len(dim))

IntegrityError: (1048, "Column '시군구명' cannot be null")

In [ ]:
# “동까지 있는 행”만 남기고 dim에 넣기

# 법정동명이 최소 3토큰(시도/구/동)을 가진 행만 필터하면 끝.

In [22]:
import pandas as pd
import numpy as np
import pymysql

path = r"C:/Users/user/Documents/서울시_포폴/법정동코드 전체자료.txt"

# 1) 단일 컬럼(탭 포함)로 읽힌 상태에서 강제 분리
raw = pd.read_csv(path, encoding="cp949")
col0 = raw.columns[0]

df = raw[col0].astype(str).str.split("\t", expand=True)
df.columns = ["법정동코드", "법정동명", "폐지여부"]
df.columns = df.columns.str.strip()

# 2) 기본 정리
df["법정동코드"] = pd.to_numeric(df["법정동코드"], errors="coerce")
df["법정동명"] = df["법정동명"].astype(str).str.strip()
df["폐지여부"] = df["폐지여부"].astype(str).str.strip()

df = df.dropna(subset=["법정동코드", "법정동명"])
df["법정동코드"] = df["법정동코드"].astype("int64")

# 3) 서울만 + 폐지 제외
df = df[df["법정동명"].str.startswith("서울특별시")]
df = df[df["폐지여부"] != "폐지"]

# 4) ✅ (핵심) 시/구/동 토큰 3개 이상인 행만 유지
tokens = df["법정동명"].str.split(r"\s+")
df = df[tokens.str.len() >= 3].copy()

# 5) 시도/구/동 분리 (이제 None 거의 안 생김)
parts = df["법정동명"].str.split(r"\s+", expand=True)
df["시도명"] = parts[0]
df["시군구명"] = parts[1]
df["행정동명"] = parts[2]

# 6) 10자리 → 8자리 행정동코드 생성
df["행정동코드"] = df["법정동코드"].astype(str).str.slice(0, 8).astype("int64")

# 7) dim 생성
dim = df[["행정동코드", "법정동코드", "시도명", "시군구명", "행정동명"]].copy()
dim["full_name"] = dim["시도명"] + " " + dim["시군구명"] + " " + dim["행정동명"]

# 8자리 기준 중복 제거
dim = dim.drop_duplicates(subset=["행정동코드"]).copy()

# ✅ 마지막 안전망: 필수 컬럼 NULL 제거 (PK/NOT NULL 보호)
dim = dim.dropna(subset=["행정동코드", "법정동코드", "시도명", "시군구명", "행정동명"])

print("dim rows:", len(dim))
print(dim.head())

# 8) MySQL insert (NULL 안전)
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="*******!",
    db="seoul",
    port=3306,
    charset="utf8mb4",
    autocommit=False
)

sql = """
INSERT INTO dim_admin_dong
(행정동코드, 법정동코드, 시도명, 시군구명, 행정동명, full_name)
VALUES (%s,%s,%s,%s,%s,%s)
ON DUPLICATE KEY UPDATE
  법정동코드=VALUES(법정동코드),
  시도명=VALUES(시도명),
  시군구명=VALUES(시군구명),
  행정동명=VALUES(행정동명),
  full_name=VALUES(full_name)
"""

data = dim.astype(object).where(pd.notnull(dim), None).values.tolist()

with conn.cursor() as cur:
    cur.executemany(sql, data)

conn.commit()
conn.close()

print("✅ dim_admin_dong 적재 완료:", len(dim))

dim rows: 467
      행정동코드       법정동코드    시도명 시군구명 행정동명      full_name
2  11110101  1111010100  서울특별시  종로구  청운동  서울특별시 종로구 청운동
3  11110102  1111010200  서울특별시  종로구  신교동  서울특별시 종로구 신교동
4  11110103  1111010300  서울특별시  종로구  궁정동  서울특별시 종로구 궁정동
5  11110104  1111010400  서울특별시  종로구  효자동  서울특별시 종로구 효자동
6  11110105  1111010500  서울특별시  종로구  창성동  서울특별시 종로구 창성동
✅ dim_admin_dong 적재 완료: 467


In [ ]:
df = pd.read_csv("C:/Users/user/Documents/서울시_포폴/법정동코드 전체자료.txt", encoding="cp949")

In [23]:
import pandas as pd

path = r"C:/Users/user/Documents/서울시_포폴/행정동_코드_김홍시_.xlsx"

df = pd.read_excel(path)
df.head()

,행정동코드,시도명,시군구명,읍면동명
0,11110515,서울특별시,종로구,청운효자동
1,11110530,서울특별시,종로구,사직동
2,11110540,서울특별시,종로구,삼청동
3,11110550,서울특별시,종로구,부암동
4,11110560,서울특별시,종로구,평창동


In [24]:
import pandas as pd
import pymysql

# ===== 1. 엑셀 읽기 =====
path = r"C:/Users/user/Documents/서울시_포폴/행정동_코드_김홍시_.xlsx"  # 실제 파일명으로 수정

df = pd.read_excel(
    path,
    dtype={"행정동코드": str}   # 코드 깨짐 방지
)

df.columns = df.columns.str.strip()

# 컬럼명 표준화
df = df.rename(columns={
    "읍면동명": "행정동명"
})

df.head()

,행정동코드,시도명,시군구명,행정동명
0,11110515,서울특별시,종로구,청운효자동
1,11110530,서울특별시,종로구,사직동
2,11110540,서울특별시,종로구,삼청동
3,11110550,서울특별시,종로구,부암동
4,11110560,서울특별시,종로구,평창동


In [25]:
# ===== 2. 전처리 =====
df = df.copy()

# 행정동코드 → int (8자리 유지)
df["행정동코드"] = pd.to_numeric(df["행정동코드"], errors="coerce")
df = df.dropna(subset=["행정동코드", "시도명", "시군구명", "행정동명"])
df["행정동코드"] = df["행정동코드"].astype("int64")

# 문자열 정리
for c in ["시도명", "시군구명", "행정동명"]:
    df[c] = df[c].astype(str).str.strip()

# 서울만 (안전장치)
df = df[df["시도명"] == "서울특별시"]

# full_name 생성
df["full_name"] = (
    df["시도명"] + " " +
    df["시군구명"] + " " +
    df["행정동명"]
)

# 중복 제거 (행정동코드 기준)
df = df.drop_duplicates(subset=["행정동코드"])

print("rows:", len(df))
df.head()

rows: 426


,행정동코드,시도명,시군구명,행정동명,full_name
0,11110515,서울특별시,종로구,청운효자동,서울특별시 종로구 청운효자동
1,11110530,서울특별시,종로구,사직동,서울특별시 종로구 사직동
2,11110540,서울특별시,종로구,삼청동,서울특별시 종로구 삼청동
3,11110550,서울특별시,종로구,부암동,서울특별시 종로구 부암동
4,11110560,서울특별시,종로구,평창동,서울특별시 종로구 평창동


## 데이터 적재

In [26]:
# ===== 3. MySQL INSERT =====
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="Abcd1234!",
    db="seoul",
    port=3306,
    charset="utf8mb4",
    autocommit=False
)

sql = """
INSERT INTO dim_admin_dong
(행정동코드, 시도명, 시군구명, 행정동명, full_name)
VALUES (%s,%s,%s,%s,%s)
ON DUPLICATE KEY UPDATE
  시도명=VALUES(시도명),
  시군구명=VALUES(시군구명),
  행정동명=VALUES(행정동명),
  full_name=VALUES(full_name)
"""

data = (
    df[["행정동코드","시도명","시군구명","행정동명","full_name"]]
      .astype(object)
      .where(pd.notnull(df), None)
      .values.tolist()
)

with conn.cursor() as cur:
    cur.executemany(sql, data)

conn.commit()
conn.close()

print("✅ dim_admin_dong 적재 완료:", len(df))

✅ dim_admin_dong 적재 완료: 426
